# Import

In [17]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers

import re
import os
from loguru import logger

import matplotlib.pyplot as plt 
%matplotlib qt

from collections import defaultdict
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
from impedance.validation import linKK       # Lin-KK (Subtractive KK) 方法&#8203;:contentReference[oaicite:0]{index=0}
import deareis                                # DearEIS (Z-HIT) 库&#8203;:contentReference[oaicite:1]{index=1}
from deareis import ZHITSettings, DataSet


In [18]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [19]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,3]
        _Zimag  = _data[:,4]
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

In [20]:
def EIS_recal_ver02(data, _phz_0 = None):
    f_poi = data[0,:]
    # Z_poi = data[1,:] * np.exp(1j*np.deg2rad(data[2,:]))
    Z_poi = data[1,:] + 1j*data[2,:]
    Y_poi = 1/Z_poi

    Rg0 = 1.611e13
    Cp0 = 1.4e-9
    
    _Rg0_rescale = 1/Rg0*np.power(f_poi,1.583)
    _Cp0_rescale = Cp0*np.power(f_poi,0.911)
    Y_org = Y_poi - _Rg0_rescale + 1j*_Cp0_rescale
    # Y_org = Y_poi - _Rg0_rescale 
    # Y_org = Y_poi + 1j*_Cp0_rescale
    # Y_org = Y_poi
    Z_org = 1/Y_org

    # Phz Calibration
    if _phz_0 is None:
        _phz_0 = np.loadtxt("./phz_Calib.txt")
    
    Z_ampC = np.abs(Z_org)
    # Z_phzC = np.angle(Z_org) - _phz_0
    Z_phzC = np.angle(Z_org) - _phz_0

    Z_rec = Z_ampC * np.exp(1j*Z_phzC)

    # C = 5e-10
    Rs0 = 100
    Z_rec = Z_rec - Rs0



    Cp0 = 5e-10
    _Cp0_rescale = Cp0 * f_poi
    Z_rec = 1/(1/Z_rec - 1j * _Cp0_rescale)

    

    # Ls0 = 1.7e-4
    Ls0 = 5e-4
    _Ls0_rescale = Ls0 * f_poi
    Z_rec = Z_rec - 1j * _Ls0_rescale

    # C = 5e-10
    Rs0 = 566
    Z_rec = Z_rec - Rs0
    
    return np.stack([f_poi, np.real(Z_rec), np.imag(Z_rec)], axis=1).T
    

# Input layer

In [21]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Archive/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

# rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
# ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/06017758_归档"
# ch_id = 96     # Perfect of Perfect

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run With Error


freq_list = np.linspace(0, 5000-1,101,dtype=int)
EISDict = gatherCSV(rootPath)


2025-05-06 16:19:46.626 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241022_01
2025-05-06 16:19:46.627 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-06 16:19:46.628 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2


2025-05-06 16:19:46.628 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-06 16:19:46.629 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-06 16:19:46.630 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241024_01
2025-05-06 16:19:46.630 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-06 16:19:46.631 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-06 16:19:46.632 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-06 16:19:46.633 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-06 16:19:46.633 | INFO     | __main__:gatherCSV:32 - Session Begin: 09290511_20241028_01
2025-05-06 16:19:46.634 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-05-06 16:19:46.634 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-05-06 16:19:46.635 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-05-06 16:19:46.635 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-05-06 16:19:46.636 | INFO     | __main__:gatherCSV:

In [22]:

chData = readChannel(ch_id, EISDict)
freq_list = np.linspace(1000,np.shape(chData)[2]-1,101,dtype=int, endpoint=True)

if True:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData)[0]):
        ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        chData[i,:,:] = ch_eis
chData = chData[:,:,freq_list]


# Run

In [25]:
def zhit_test(data, settings=None):
    """
    对 EIS 数据执行 Z-HIT 测试，并绘制重建误差。

    Parameters
    ----------
    data : ndarray, shape (3, N)
        row 0: frequency (Hz)
        row 1: impedance magnitude (Ω)
        row 2: phase (degrees)
    settings : deareis.ZHITSettings, optional
        Z-HIT 参数；默认使用窗口中心 1.5（≈30 Hz），宽度 3 个 decade

    Returns
    -------
    None  # 在 Matplotlib 窗口中绘制 Bode 误差图
    """
    # 准备 DataSet
    f = data[0]
    # Z = data[1] * np.exp(1j*np.deg2rad(data[2]))
    Z = data[1] + 1j*data[2]
    ds = DataSet(frequencies=f, impedances=Z)

    # 默认 ZHITSettings
    # if settings is None:
    #     settings = ZHITSettings(
    #         smoothing=ZHITSettings.smoothing.AUTO,
    #         num_points=5, polynomial_order=3, num_iterations=2,
    #         interpolation=ZHITSettings.interpolation.AUTO,
    #         window=ZHITSettings.window.BOXCAR,
    #         window_center=1.5, window_width=3.0,
    #         representation='impedance'
    #     )
    if settings is None:
        settings = ZHITSettings(
            smoothing=deareis.ZHITSmoothing.NONE,
            num_points=5, polynomial_order=3, num_iterations=2,
            interpolation=deareis.ZHITInterpolation.AUTO,
            window=deareis.ZHITWindow.BOXCAR,
            window_center=1.5, window_width=3.0,
            representation=deareis.ZHITRepresentation.IMPEDANCE
        )

    # 执行 Z-HIT
    result = deareis.perform_zhit(ds, settings)  # :contentReference[oaicite:2]{index=2}

    # 获取残差并绘图
    _freq, res_real, res_imag = result.get_residuals_data()
    _freq, amp_HIT, phz_HIT = result.get_bode_data()


    fig, axis = plt.subplots(3,1)

    axis[0].semilogx(_freq, res_real, 's-', label='Real error')
    axis[0].semilogx(_freq, res_imag, 'o-', label='Imag error')
    axis[0].set_xlabel('Frequency (Hz)')
    axis[0].set_ylabel('Residual (Ω)')
    # axis[0].set_ylim(-3,3)
    axis[0].grid(True)

    axis[1].loglog(f, np.abs(Z), 'b')
    axis[1].loglog(_freq, amp_HIT, 'r.')

    
    axis[2].semilogx(f, -np.rad2deg(np.angle(Z)), 'b')
    axis[2].semilogx(_freq, phz_HIT, 'r.')

    # plt.title('Z-HIT Residuals')
    # plt.legend()
    # plt.show()


def subtractive_kk_test(data, f0=None):
    """
    对 EIS 数据执行 Subtractive Kramers–Kronig 测试，并绘制重建误差。

    Uses:
      Z_real_calc(w) = Z_real(f0) + (2/π) ∫ [x Z_imag(x) - w Z_imag(w)] / (x^2 - w^2) dx

    Parameters
    ----------
    data : ndarray, shape (3, N)
        row 0: frequency (Hz)
        row 1: impedance magnitude (Ω)
        row 2: phase (degrees)
    f0 : float, optional
        参考频率；默认取频率数组中值

    Returns
    -------
    None  # 绘制 Subtractive KK 实部重建误差
    """
    f = data[0]
    # Z = data[1] * np.exp(1j*np.deg2rad(data[2]))
    # Zr, Zi = Z.real, Z.imag
    Zr = data[1]  
    Zi = data[2]

    # 参考频率
    if f0 is None:
        f0 = f[len(f)//2]
    Z0 = np.interp(f0, f, Zr)

    # 对每个 w 计算积分
    Zr_calc = np.zeros_like(Zr)
    for i, w in enumerate(f):
        # 跳过 x == w 点
        mask = f != w
        x = f[mask]
        Zi_x = Zi[mask]
        integrand = (x * Zi_x - w * Zi[i]) / (x**2 - w**2)
        Zr_calc[i] = Z0 + (2/np.pi) * np.trapz(integrand, x)

    # 绘制误差
    err = (Zr - Zr_calc) / Zr_calc * 100  # 相对误差
    plt.figure()
    plt.semilogx(f, err, 'o-')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Real-part residual (%)')
    plt.title('Subtractive KK Residuals')
    plt.grid(True)
    plt.show()


In [31]:

chData = readChannel(ch_id, EISDict)
freq_list = np.linspace(1000,np.shape(chData)[2]-1,101,dtype=int, endpoint=True)

if True:
    phz_calibration = np.loadtxt("./phz_Calib.txt")
    for i in range(np.shape(chData)[0]):
        ch_eis = EIS_recal_ver02(chData[i,:,:], phz_calibration)
        chData[i,:,:] = ch_eis
chData = chData[:,:,freq_list]


zhit_test(chData[-3,:,:])